## Machine Learning

- Para nuestro Sistema de Recomendacion utilizamos los filtros basados en contenido (contend-based filtering) como base de la prediccion, es decir, que sugiere películas similares basados en una películas en particular.

- En este sistema, las caracteristicas de las películas (overview, genre, company, actor) se usan para encontrar su similitud con otras películas. Luego se recomiendan 5 películas que tienen más probabilidades de ser similares a la consultada en nuestra API.

- El procesamiento del lenguaje natural (NLP) es un proceso de manipulación o comprensión del texto por cualquier software o máquina.

- Utilizamos La biblioteca NLTK (Natural Language Toolkit), la cual es una suite que contiene bibliotecas para el procesamiento estadístico del lenguaje, y de esta forma hacer que las máquinas entiendan el lenguaje humano y respondan con una respuesta adecuada.

- También utilizamos RAKE, el cual es un algoritmo de extracción de palabras clave en un cuerpo de texto mediante el análisis de la frecuencia de aparición de palabras y su co-ocurrencia con otras palabras en el texto.

- Utilizamos el Stemming, es una técnica de normalización de texto en el procesamiento del lenguaje natural. Esta técnica es ampliamente utilizadas para el preprocesamiento de texto. Es una técnica en la que un conjunto de palabras en una oración se convierten en una secuencia para acortar su búsqueda. En este método, se normalizan las palabras que tienen el mismo significado pero tienen algunas variaciones según el contexto o la oración.

-  Para nuestro modelo utilizamos la técnica k-Nearest Neighbor (**Knn**), mejor conocido como el método de vecinos más cercanos, para definir las recomendaciones, es un método de clasificación no paramétrico, la cual determina cuán similares son los objetos de datos independientemente de su tamaño.

-  Para calcular los (**Knn**), necesitamos el número de palabras de cada observacion de la columna 'tags'. Para realizar estos cálculos utilizamos CountVectorizer de Sklearn para aprender el vocabulario del conjunto de textos y luego transformarlos en un marco de datos que utilizamos para construir el modelo. Este proceso nos da como salida una matriz de correlaciones, luego define una matriz de distancia por medio de la matriz de correlaciones, y se aplica el método Knn, el cual realizará  la estimación.

-  Adicionalmente creamos una serie con los índices de cada una de de las películas , la cual utilizaremos en nuestra estimacion.

-  En nuestra función de recomendación, pasamos como parámetros el titulo de la películas que deseamos consular. Se realiza la búsqueda del titulo en el archivo de índices, el índice de la misma es buscado por (**Knn**), se obtiene una matriz la cual permite conocer cuales son las películas con mayor probabilidad para poder recomendarlas al usuario. La última línea muestra las 5 películas con mayor probabilidad de recomendar al usuario 1. Si la película no se encuentra en el set de datos, retorna una notificación.



1 - Importamos las librerias necesarias

In [1]:
import pandas as pd
import rake_nltk
from rake_nltk import Rake
import nltk #Natural Language ToolKit
nltk.download('stopwords')
nltk.download('punkt')
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer #para convertir las caracteristicas combinadas en una matriz
from sklearn.neighbors import NearestNeighbors

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\58424\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\58424\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
#Importamos el dataset
movies = pd.read_csv('datasets/movies.csv', sep=';', encoding='utf-8')

2 - Utilizamos la libreria Rake para extraer las palabras claves de cada película, de la columna 'overview'. Creamos la columna 'words' para almacenar las palabras extraídas.

In [3]:
movies['words'] = ""

for index, row in movies.iterrows():
    r = Rake()  #Instancia de Rake
    overview = row['overview']
    r.extract_keywords_from_text(overview)  #Metodo para extraer palabras
    score_words = r.get_word_degrees()    #Diccionario con las palabras claves y sus puntuaciones
    movies.at[index, 'words'] = list(score_words.keys()) #Asignamos las palabras claves a la columna words

In [4]:
movies['genres'] = movies['genre'].str.split(',')
movies['companies'] = movies['company'].str.split(',')
movies['actors'] = movies['actor'].str.split(',')

#Remplazamos los valos " " por "" en cada una de las listas de las columnas desanidadas
movies['genres'] = movies['genres'].apply(lambda x: [i.replace(" ", "") for i in x])
movies['companies'] = movies['companies'].apply(lambda x: [i.replace(" ", "") for i in x])
movies['actors'] = movies['actors'].apply(lambda x: [i.replace(" ", "") for i in x])

#Creamos una columna que reuna los datos de las columnas que usamos para alimentar el modelo
movies['tags'] = movies['words'] + movies['genres'] + movies['companies'] + movies['actors']

# Se dejan unicamente la columna 'tags' en el modelo puesto que es la unica que se va a ausar para la alimentacion del modelo
moviesML = movies[['title','tags']]

# unimos todas las cadenas en una sola, con respecto a la columna 'tags'
moviesML['tags'] = moviesML['tags'].apply(lambda x:' '.join(x))

C:\Users\58424\AppData\Local\Temp\ipykernel_17876\4184650190.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  moviesML['tags'] = moviesML['tags'].apply(lambda x:' '.join(x))


3 - Utilizamos el Stemming, es una técnica de normalización de texto en el procesamiento del lenguaje natural. 

In [5]:
# Implementacion de la libreria para normalizar la columna 'tags'
ps = PorterStemmer()

def normalization(text):
    tag = []
    for i in text.split():
        tag.append(ps.stem(i))
    return " ".join(tag)

# aplicamos la funcion sobre la columana 'tags'
moviesML['tags'] = moviesML['tags'].apply(normalization)

C:\Users\58424\AppData\Local\Temp\ipykernel_17876\4101395328.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  moviesML['tags'] = moviesML['tags'].apply(normalization)


4 - Exportamos a un archivo .csv

In [6]:
moviesML.to_csv('datasets/moviesML.csv', index=True)

5 - Para calcular los (**Knn**), necesitamos el número de palabras de cada observacion de la columna 'tags'. Para realizar estos cálculos utilizamos CountVectorizer de Sklearn para aprender el vocabulario del conjunto de textos

In [7]:
count_vectorizer  = CountVectorizer()
sparse_matrix  = count_vectorizer.fit_transform(moviesML['tags'])

# Creamos un modelo para encontrar los vecinos mas cercanos en un espacio de caracterisicaa
nn = NearestNeighbors(metric='cosine', algorithm='brute')
nn.fit(sparse_matrix)

indicesML = pd.Series(moviesML.index, index=moviesML['title']).drop_duplicates()

6 - Función de Recomendación

In [8]:
def recomendacion(titulo):

    title=titulo.lower()

    #Validar que el titulo ingresado se encuentra en el dataframe
    if title not in moviesML['title'].values:
        return 'Película no encontrada! Prueba con: Star Wars, Back to the Future, Dracula, Cars, Batman, Superman, Toy Story'
    else:
        # Si el título esta en el df, encuentra su indice
        index = indicesML[title]

        # Obtiene las puntuaciones de similitud de las 5 peliculas más cercanas
        distances, indices_knn = nn.kneighbors(sparse_matrix[index], n_neighbors=6)  # indica que queremos encontrar las 6 peliculas más similares, incluyendo la pelicula dada

        # Obtiene los indices de las peliculas
        movie_indices = indices_knn[0][1:]  # Se omite el primer indice (la pelicula misma) con [1:]

        # Devuelve las 5 peliculas mas similares
        return moviesML['title'].iloc[movie_indices].tolist()

In [9]:
recomendacion('cars')

['cars 3', 'cars 2', "a bug's life", 'monsters, inc.', 'monsters university']

In [10]:
recomendacion('batman')

['batman & robin',
 'batman forever',
 'batman returns',
 'the lego batman movie',
 'batman: mask of the phantasm']

In [11]:
recomendacion('dracula')

['dracula 2000',
 'van helsing',
 "it's alive",
 "mary shelley's frankenstein",
 'vampire in brooklyn']

In [12]:
recomendacion('jurassic park')

['jurassic world',
 'the lost world: jurassic park',
 'scooby-doo',
 'zathura: a space adventure',
 'rumble fish']

In [13]:
recomendacion('superman')

['superman ii',
 'superman iv: the quest for peace',
 'superman iii',
 'superman returns',
 'max steel']

In [14]:
recomendacion('toy story')

['toy story 2', 'toy story 3', 'free birds', "a bug's life", 'monsters, inc.']

In [15]:
recomendacion('hotel transylvania')

['hotel transylvania 2',
 'grown ups',
 'monster house',
 'grown ups 2',
 'the croods']

In [16]:
recomendacion('star wars')

['the empire strikes back',
 'return of the jedi',
 'star wars: the force awakens',
 'star wars: episode iii - revenge of the sith',
 'star wars: episode i - the phantom menace']

In [17]:
recomendacion('transformers')

['transformers: revenge of the fallen',
 'transformers: dark of the moon',
 'transformers: age of extinction',
 'transformers: the last knight',
 'double dragon']

In [18]:
recomendacion('back to the future')

['back to the future part ii',
 'back to the future part iii',
 'spacecamp',
 'honey, i shrunk the kids',
 'hot tub time machine']